In [1]:
import numpy as np
from PIL import Image

In [2]:

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import argparse
import time
import numpy as np
import collections

from tensorboardX import SummaryWriter

In [3]:
import gym

env = gym.make("FetchReach-v1")
DEFAULT_ENV_NAME="FetchReach-v1"

/home/andrew/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
print(env.action_space)
print(env.action_space.sample())

Box(4,)
[0.09762701 0.43037874 0.20552675 0.08976637]


In [ ]:
print(env.observation_space)

Dict(achieved_goal:Box(3,), desired_goal:Box(3,), observation:Box(10,))


In [ ]:
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        # 4 input image channel, 32 output channels, 3x3 square convolution
        # kernel
        self.pipe = nn.Sequential(
            nn.Linear(16,128),
            nn.ReLU(),
            nn.Linear(128,256),
            nn.ReLU(),
            nn.Linear(256,4)
        )

    def forward(self, x):  
        x= x.float()
        return self.pipe(x)


In [ ]:
GAMMA = 0.99
BATCH_SIZE = 32
REPLAY_SIZE = 10000
REPLAY_START_SIZE = 10000
LEARNING_RATE = 1e-4
SYNC_TARGET_FRAMES = 1000

In [ ]:
#epsilon decay, for choosing random actions to explore space
EPSILON_DECAY_LAST_FRAME = 10**5
EPSILON_START = 1.0
EPSILON_FINAL = 0.02
#Going from 1 to .02 in 100000 frames
MEAN_REWARD_BOUND=50

In [ ]:
#define what experience is
Experience = collections.namedtuple("Experience", 
                        field_names = ['state','action','reward', 'done','new_state'])

class ExperienceBuffer:
     #This acts as our replay buffer
    def __init__(self,capacity):
        self.buffer = collections.deque(maxlen=capacity) #rotating buffer
    
    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)
        
    def sample(self,batch_size):
        indices = np.random.choice(
            len(self.buffer), batch_size, replace =False)
        
        states,actions,rewards,dones, next_states = \
            zip(*[self.buffer[idx] for idx in indices])
        
        states,actions = np.array(states),np.array(actions)
        rewards = np.array(rewards,dtype=np.float32)
        dones =np.array(dones, dtype=np.float32)
        return states,actions,rewards,dones,next_states

In [ ]:
#make our agent that acts and learns in the environment
class Agent:
    def __init__(self, env, exp_buffer):
        self.env = env
        self.exp_buffer = exp_buffer
        self._reset()
        
    def _reset(self):
        state = env.reset()
        
        
        self.state = np.concatenate(list(state.values()))
        self.total_reward = 0.0
    
    def play_step(self, net, epsilong=0.0, device ='cpu'):
        done_reward = None
        
        if np.random.random() < epsilon:
            action = env.action_space.sample()
        else:
            state_a = np.array([self.state], copy=False)
            state_v = torch.tensor(state_a).to(device)
            q_vals_v =  net(state_v)
            _, act_v = torch.max(q_vals_v,dim=1)
            
            #print("whats going on here?")
            action = int(act_v.item())
            
            #take the action and update
            #env.show()
            new_state, reward, is_done, _ = self.env.step(action)  
            new_state = np.concatenate(list(new_state.values()))
            reward +=1.0
            self.total_reward += reward
            #do we need this? is it doing a copy?
            new_state = new_state
            
            exp = Experience(self.state,action,reward,is_done,new_state)
            
            self.exp_buffer.append(exp)
            self.state = new_state
            
            if(is_done):
                done_reward = self.total_reward
                self._reset()
            return done_reward
        
def calc_loss(batch, net, target_net, device='cpu'):
    states,actions,rewards,dones,next_states = batch
    
    
    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.ByteTensor(dones).to(device)

    #predictions
    #look into gather further
    state_action_values = net(states_v).gather(1,actions_v.unsqueeze(-1)).squeeze(-1)

    #what is the value of being in the state that our action took us to?
    next_state_values = target_net(next_states_v).max(1)[0] #take the max along the first axis


    # saying the reward of the step after finishing is zero
    # required to converege
    next_state_values[done_mask] = 0.0

    #we do not want to perform backprop on this 
    next_state_values = next_state_values.detach() 

    #-------------Bellman equation------------------
    expected_state_action_values = next_state_values * GAMMA + rewards_v

    return nn.MSELoss()(state_action_values, expected_state_action_values)

In [ ]:
if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument("--cude", default=False, action='store_true', help="Enable cuda")
    
    #parser.add_argument("--env", default= DEFAULT_ENV_NAME, 
    #                    help="name of the environment, default="+DEFAULT_ENV_NAME)
    
    #parser.add_argument("--reward", type=float, default=MEAN_REWARD_BOUND, 
    #                    help="Mean reward boundary for stop of training, default= %.2f"% MEAN_REWARD_BOUND)
    
    #args = parser.parse_args()
    args= [False,DEFAULT_ENV_NAME,MEAN_REWARD_BOUND]
    #device= torch.device("cuda" if args.cuda else "cpu")
    device = 'cpu'
    
    env = gym.make(DEFAULT_ENV_NAME)#args.env)
    net = DQN().to(device)
    
    target_net = DQN().to(device)
    
    writer = SummaryWriter(comment='-'+DEFAULT_ENV_NAME)#args.env)
    print(net)
    
    buffer = ExperienceBuffer(REPLAY_SIZE)
    agent = Agent(env, buffer)
    
    epsilon = EPSILON_START
    
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    
    best_mean_reward = None
    
    while True:
        frame_idx += 1
        epsilon = max(EPSILON_FINAL, EPSILON_START - (frame_idx / EPSILON_DECAY_LAST_FRAME))
        
        reward = agent.play_step(net, epsilon, device=device)
        if reward is not None:
            total_rewards.append(reward)
            speed = (frame_idx - ts_frame) / (time.time() - ts)
            ts_frame = frame_idx
            ts = time.time()
            
            mean_reward = np.mean(total_rewards[-100:])
            print("%d: done %d games, mean reward %.3f, eps %.2f, speed %.2f f/s" %(
                frame_idx, len(total_rewards), mean_reward, epsilon, speed
            ))
            
            writer.add_scalar("epsilon", epsilon, frame_idx)
            writer.add_scalar("speed", speed, frame_idx)
            writer.add_scalar("reward_100", mean_reward, frame_idx)
            writer.add_scalar("reward", reward, frame_idx)
            
            if best_mean_reward  is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), DEFAULT_ENV_NAME+"-best.dat")
                if(best_mean_reward is not None):
                    print('best mean reward updated %.3f -> %.3f, model saved'%
                             (best_mean_reward, mean_reward))
                    best_mean_reward = mean_reward
                if(mean_reward > MEAN_REWARD_BOUND):
                    print("solved in %d frames!" % frame_idx)
                    break
                
            if(len(buffer) < REPLAY_START_SIZE):
                continue
            
            if(frame_idx % SYNC_TARGET_FRAMES == 0):
                target_net.load_state_dict(net.state_dict())
            
            optimizer.zero_grad()
            batch = buffer.sample(BATCH_SIZE)
            loss_t = calc_loss(batch, net, target_net, device=device)
            
            loss_t.backward()
            optimizer.step()

DQN(
  (pipe): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=4, bias=True)
  )
)
3718: done 1 games, mean reward 0.000, eps 0.96, speed 21183.70 f/s
4767: done 2 games, mean reward 0.500, eps 0.95, speed 9019.97 f/s
5606: done 3 games, mean reward 0.333, eps 0.94, speed 8220.09 f/s
6450: done 4 games, mean reward 0.250, eps 0.94, speed 8127.77 f/s
7146: done 5 games, mean reward 0.400, eps 0.93, speed 6285.20 f/s
7945: done 6 games, mean reward 0.333, eps 0.92, speed 6894.99 f/s
8480: done 7 games, mean reward 0.429, eps 0.92, speed 4919.51 f/s
9052: done 8 games, mean reward 0.375, eps 0.91, speed 5485.77 f/s
9647: done 9 games, mean reward 0.333, eps 0.90, speed 5484.47 f/s
10036: done 10 games, mean reward 0.500, eps 0.90, speed 3993.85 f/s
10431: done 11 games, mean reward 0.455, eps 0.90, speed 3693.94 f/s
10886: don

34638: done 119 games, mean reward 0.110, eps 0.65, speed 1617.72 f/s
34776: done 120 games, mean reward 0.110, eps 0.65, speed 1499.37 f/s
34952: done 121 games, mean reward 0.110, eps 0.65, speed 1806.24 f/s
35122: done 122 games, mean reward 0.110, eps 0.65, speed 1584.34 f/s
35245: done 123 games, mean reward 0.110, eps 0.65, speed 1270.93 f/s
35391: done 124 games, mean reward 0.110, eps 0.65, speed 1590.61 f/s
35534: done 125 games, mean reward 0.110, eps 0.64, speed 1276.41 f/s
35674: done 126 games, mean reward 0.110, eps 0.64, speed 1533.58 f/s
35840: done 127 games, mean reward 0.110, eps 0.64, speed 1811.77 f/s
35986: done 128 games, mean reward 0.110, eps 0.64, speed 1400.31 f/s
36108: done 129 games, mean reward 0.110, eps 0.64, speed 1342.80 f/s
36245: done 130 games, mean reward 0.110, eps 0.64, speed 1501.40 f/s
36374: done 131 games, mean reward 0.110, eps 0.64, speed 1231.70 f/s
36478: done 132 games, mean reward 0.120, eps 0.64, speed 1141.56 f/s
36609: done 133 game

48826: done 237 games, mean reward 0.040, eps 0.51, speed 958.98 f/s
48919: done 238 games, mean reward 0.040, eps 0.51, speed 652.32 f/s
49016: done 239 games, mean reward 0.040, eps 0.51, speed 897.57 f/s
49104: done 240 games, mean reward 0.040, eps 0.51, speed 810.43 f/s
49195: done 241 games, mean reward 0.040, eps 0.51, speed 547.34 f/s
49298: done 242 games, mean reward 0.040, eps 0.51, speed 1034.37 f/s
49397: done 243 games, mean reward 0.040, eps 0.51, speed 1024.81 f/s
49505: done 244 games, mean reward 0.030, eps 0.50, speed 1019.94 f/s
49616: done 245 games, mean reward 0.030, eps 0.50, speed 1009.43 f/s
49722: done 246 games, mean reward 0.030, eps 0.50, speed 978.85 f/s
49822: done 247 games, mean reward 0.030, eps 0.50, speed 772.86 f/s
49924: done 248 games, mean reward 0.030, eps 0.50, speed 955.25 f/s
50021: done 249 games, mean reward 0.030, eps 0.50, speed 851.98 f/s
50124: done 250 games, mean reward 0.030, eps 0.50, speed 1035.88 f/s
50218: done 251 games, mean r

59706: done 356 games, mean reward 0.030, eps 0.40, speed 729.53 f/s
59790: done 357 games, mean reward 0.030, eps 0.40, speed 828.14 f/s
59870: done 358 games, mean reward 0.030, eps 0.40, speed 789.91 f/s
59953: done 359 games, mean reward 0.020, eps 0.40, speed 762.10 f/s
60023: done 360 games, mean reward 0.020, eps 0.40, speed 752.56 f/s
60110: done 361 games, mean reward 0.020, eps 0.40, speed 834.28 f/s
60208: done 362 games, mean reward 0.020, eps 0.40, speed 1015.18 f/s
60291: done 363 games, mean reward 0.020, eps 0.40, speed 771.84 f/s
60373: done 364 games, mean reward 0.020, eps 0.40, speed 788.97 f/s
60459: done 365 games, mean reward 0.020, eps 0.40, speed 871.92 f/s
60549: done 366 games, mean reward 0.020, eps 0.39, speed 689.78 f/s
60629: done 367 games, mean reward 0.020, eps 0.39, speed 756.34 f/s
60709: done 368 games, mean reward 0.020, eps 0.39, speed 691.34 f/s
60795: done 369 games, mean reward 0.020, eps 0.39, speed 806.34 f/s
60871: done 370 games, mean rewar

69032: done 477 games, mean reward 0.060, eps 0.31, speed 609.43 f/s
69106: done 478 games, mean reward 0.060, eps 0.31, speed 753.21 f/s
69172: done 479 games, mean reward 0.060, eps 0.31, speed 606.63 f/s
69244: done 480 games, mean reward 0.060, eps 0.31, speed 485.02 f/s
69315: done 481 games, mean reward 0.060, eps 0.31, speed 544.94 f/s
69387: done 482 games, mean reward 0.060, eps 0.31, speed 599.03 f/s
69466: done 483 games, mean reward 0.060, eps 0.31, speed 762.37 f/s
69534: done 484 games, mean reward 0.060, eps 0.30, speed 622.35 f/s
69598: done 485 games, mean reward 0.060, eps 0.30, speed 651.79 f/s
69675: done 486 games, mean reward 0.060, eps 0.30, speed 709.58 f/s
69754: done 487 games, mean reward 0.060, eps 0.30, speed 808.02 f/s
69832: done 488 games, mean reward 0.060, eps 0.30, speed 659.91 f/s
69903: done 489 games, mean reward 0.060, eps 0.30, speed 655.72 f/s
69971: done 490 games, mean reward 0.060, eps 0.30, speed 698.39 f/s
70043: done 491 games, mean reward

...

211149: done 3179 games, mean reward 0.060, eps 0.02, speed 525.59 f/s
211199: done 3180 games, mean reward 0.060, eps 0.02, speed 500.66 f/s
211250: done 3181 games, mean reward 0.060, eps 0.02, speed 537.74 f/s
211301: done 3182 games, mean reward 0.060, eps 0.02, speed 534.21 f/s
211354: done 3183 games, mean reward 0.060, eps 0.02, speed 558.34 f/s
211405: done 3184 games, mean reward 0.060, eps 0.02, speed 488.45 f/s
211458: done 3185 games, mean reward 0.060, eps 0.02, speed 513.08 f/s
211508: done 3186 games, mean reward 0.060, eps 0.02, speed 539.20 f/s
211559: done 3187 games, mean reward 0.060, eps 0.02, speed 555.72 f/s
211609: done 3188 games, mean reward 0.060, eps 0.02, speed 538.35 f/s
211659: done 3189 games, mean reward 0.060, eps 0.02, speed 543.62 f/s
211709: done 3190 games, mean reward 0.060, eps 0.02, speed 543.33 f/s
211762: done 3191 games, mean reward 0.070, eps 0.02, speed 536.76 f/s
211813: done 3192 games, mean reward 0.070, eps 0.02, speed 551.12 f/s
211864

217083: done 3295 games, mean reward 0.150, eps 0.02, speed 523.29 f/s
217133: done 3296 games, mean reward 0.150, eps 0.02, speed 496.47 f/s
217185: done 3297 games, mean reward 0.150, eps 0.02, speed 543.37 f/s
217236: done 3298 games, mean reward 0.150, eps 0.02, speed 533.15 f/s
217287: done 3299 games, mean reward 0.150, eps 0.02, speed 507.34 f/s
217337: done 3300 games, mean reward 0.150, eps 0.02, speed 500.69 f/s
217390: done 3301 games, mean reward 0.150, eps 0.02, speed 529.51 f/s
217442: done 3302 games, mean reward 0.150, eps 0.02, speed 559.76 f/s
217493: done 3303 games, mean reward 0.150, eps 0.02, speed 552.77 f/s
217545: done 3304 games, mean reward 0.150, eps 0.02, speed 527.57 f/s
217597: done 3305 games, mean reward 0.150, eps 0.02, speed 561.11 f/s
217647: done 3306 games, mean reward 0.150, eps 0.02, speed 495.31 f/s
217699: done 3307 games, mean reward 0.140, eps 0.02, speed 524.31 f/s
217749: done 3308 games, mean reward 0.140, eps 0.02, speed 525.47 f/s
217799

223043: done 3412 games, mean reward 0.060, eps 0.02, speed 376.14 f/s
223094: done 3413 games, mean reward 0.060, eps 0.02, speed 539.39 f/s
223145: done 3414 games, mean reward 0.060, eps 0.02, speed 540.35 f/s
223196: done 3415 games, mean reward 0.060, eps 0.02, speed 507.86 f/s
223246: done 3416 games, mean reward 0.060, eps 0.02, speed 490.10 f/s
223296: done 3417 games, mean reward 0.060, eps 0.02, speed 522.42 f/s
223348: done 3418 games, mean reward 0.060, eps 0.02, speed 522.62 f/s
223399: done 3419 games, mean reward 0.060, eps 0.02, speed 527.60 f/s
223450: done 3420 games, mean reward 0.060, eps 0.02, speed 539.89 f/s
223501: done 3421 games, mean reward 0.070, eps 0.02, speed 516.69 f/s
223554: done 3422 games, mean reward 0.070, eps 0.02, speed 481.08 f/s
223607: done 3423 games, mean reward 0.070, eps 0.02, speed 524.02 f/s
223657: done 3424 games, mean reward 0.070, eps 0.02, speed 526.05 f/s
223710: done 3425 games, mean reward 0.070, eps 0.02, speed 559.06 f/s
223760

228959: done 3528 games, mean reward 0.050, eps 0.02, speed 544.12 f/s
229011: done 3529 games, mean reward 0.050, eps 0.02, speed 547.44 f/s
229063: done 3530 games, mean reward 0.040, eps 0.02, speed 514.04 f/s
229113: done 3531 games, mean reward 0.040, eps 0.02, speed 499.97 f/s
229166: done 3532 games, mean reward 0.040, eps 0.02, speed 527.98 f/s
229217: done 3533 games, mean reward 0.040, eps 0.02, speed 512.40 f/s
229274: done 3534 games, mean reward 0.040, eps 0.02, speed 587.84 f/s
229324: done 3535 games, mean reward 0.040, eps 0.02, speed 500.58 f/s
229375: done 3536 games, mean reward 0.040, eps 0.02, speed 539.40 f/s
229427: done 3537 games, mean reward 0.050, eps 0.02, speed 547.03 f/s
229479: done 3538 games, mean reward 0.060, eps 0.02, speed 549.96 f/s
229530: done 3539 games, mean reward 0.060, eps 0.02, speed 510.42 f/s
229580: done 3540 games, mean reward 0.060, eps 0.02, speed 501.84 f/s
229631: done 3541 games, mean reward 0.060, eps 0.02, speed 508.41 f/s
229682

234999: done 3646 games, mean reward 0.080, eps 0.02, speed 518.96 f/s
235049: done 3647 games, mean reward 0.080, eps 0.02, speed 531.35 f/s
235099: done 3648 games, mean reward 0.080, eps 0.02, speed 531.54 f/s
235149: done 3649 games, mean reward 0.080, eps 0.02, speed 495.55 f/s
235200: done 3650 games, mean reward 0.060, eps 0.02, speed 536.53 f/s
235251: done 3651 games, mean reward 0.060, eps 0.02, speed 540.74 f/s
